In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

#DOWNLOAD THE DATA

from google.colab import drive
import zipfile

#MOUNT GDRIVE: DOWNLOAD DATA AND FILTERED LABELS
drive.mount('/content/gdrive',force_remount=True)

# UNZIP ZIP
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('/content/gdrive/My Drive/CheXpert-v1.0-small.zip', 'r')
zip_ref.extractall()
zip_ref.close()
print("downloaded files")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Uncompressing zip file
downloaded files


In [0]:
import sys #For Patch lib
sys.path.append("gdrive/My Drive/summerthesis/patch")

In [0]:
import os#for CUDA tracking
#from __future__ import print_function, division
import torch
import pandas as pd
from skimage.io import imread
#from skimage import io
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
#MODELS
import re
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import matplotlib.pyplot as plt

#Patch
import patch


# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

class CheXpertDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
              
        super().__init__()

        self.observations_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
        self.list_classes=['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']
        
    def __len__(self):
        return len(self.observations_frame)

    def __getitem__(self, idx):
        
        img_name = os.path.join(self.observations_frame.iloc[idx, 0])
        image = imread(img_name)
        image = transforms.ToPILImage()(image)
      

        observations=self.observations_frame.loc[idx,self.list_classes]
        observations = torch.from_numpy(observations.values.astype(np.float32))
        #np.float32()
        
     
        #RETURNING IMAGE AND LABEL SEPERATELY FOR TORCH TRANSFORM LIB
      
        if self.transform:
            
            image = self.transform(image)

        return image,observations

def chexpert_load(csv_file_name,transformation,batch_size):
  
  cheXpert_dataset = CheXpertDataset(csv_file=csv_file_name,
                                     root_dir='not used', transform=transformation)

  dataloader = DataLoader(cheXpert_dataset, batch_size=batch_size,shuffle=True)

  return cheXpert_dataset,dataloader



use_cuda = True
if use_cuda and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    print("CUDA didn't work")
    device = torch.device('cpu')



In [0]:
x=torch.ones(16,1024)

x = torch.stack([torch.cat((x[idx,:],x[idx+1,:]))
                     for idx in range(0,16,2)], dim=0)

print(x.shape)
  

  

torch.Size([8, 2048])


In [0]:
#HOW TO USE RES BLOCK AT THE END AS A Class there operations between features and classifier that causes dimension conflict
#LOSS FUNC JUST A 8 WAY SOFTMAX

class RelativePositionHead(torch.nn.Module):
    def __init__(self, D_in, D_out=8):
        """
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(RelativePositionHead, self).__init__()
        self.res_block1 = models.resnet.BasicBlock(D_in,512)
        self.res_block2 = models.resnet.BasicBlock(512,512)
        self.classifier = torch.nn.Linear(512,D_out)

    def forward(self, x):
        
        #print(x.shape)
        x = self.res_block1(x)
        x = self.res_block2(x)
        
        _,N,_,_ = x.shape
        # combining two representation (batch is [bs,ch,h,w ])
        x = torch.stack([torch.cat((x[idx,:,:,:],x[idx+1,:,:,:]))
                     for idx in range(0,N,2)], dim=0)
        
        #linear output with 8 outpts(directions)
        y_pred = self.classifier(x)
        return y_pred

      
      
class Basic_RelativePositionHead(torch.nn.Module):
    def __init__(self, D_in, D_out=8):
        """
        No task head just concating what comes out just before default classifer then applying linear
        In the constructor we instantiate two nn.Linear modules and assign them as
        member variables.
        """
        super(Basic_RelativePositionHead, self).__init__()
        self.classifier = torch.nn.Linear(D_in*2,D_out)

    def forward(self, x):
                
        N,_ = x.shape
        # combining two representation (batch is [bs,ch,h,w ])
        x = torch.stack([torch.cat((x[idx,:],x[idx+1,:]))
                     for idx in range(0,N,2)], dim=0).cuda()
        
        #linear output with 8 outpts(directions)
        y_pred = self.classifier(x)
        return y_pred
    


In [0]:
learning_rate=0.0001
batch_size=32
dtype = torch.float32
resize=160
split=2.0
num_classes=8

#just ToTensor before pathch
transform_train= transforms.Compose([          transforms.RandomCrop(320),
                                               transforms.ToTensor()])



#after patch transformation
transform_after_patch= transforms.Compose([    transforms.ToPILImage(),
                                               transforms.Resize(resize),
                                               transforms.ToTensor(),
                                               transforms.Lambda(lambda x: torch.cat([x, x, x], 0)),
                                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])                                 

cheXpert_train_dataset, dataloader = chexpert_load("CheXpert-v1.0-small/train.csv",transform_train,batch_size)



model=models.densenet121(num_classes=num_classes)
#model.classifier=RelativePositionHead(1024,num_classes)
model.classifier=Basic_RelativePositionHead(1024,num_classes)
model=model.to(device=device)


optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion =torch.nn.CrossEntropyLoss().cuda()
#criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights).cuda()

num_epochs=1
for epoch in range(num_epochs):
    for i,  (images, observations) in enumerate(dataloader):   # Load a batch of images with its (index, data, class)
        
        
        patcher = patch.Patch(images,split=split,show=False)
        patches, labels = patcher()
        
        patches = torch.stack([transform_after_patch(x_i) 
                     for i, x_i in enumerate(torch.unbind(patches, dim=0))], dim=0)
        
        
        patches = patches.to(device=device,dtype=dtype)
        labels = torch.from_numpy(labels)
        labels = labels.long()
        labels = labels.to(device=device,dtype=torch.long)
        #print("patches ",patches.shape)

        outputs = model(patches)                             # Forward pass: compute the output class given a image
        
        #print("model outputs ",outputs.shape)

        loss = criterion(outputs, labels)           # Compute the loss: difference between the output class and the pre-given label

        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        
        if (i+1) % 50 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(cheXpert_train_dataset)//batch_size, loss))

print('training done')





Epoch [1/1], Step [50/6981], Loss: 1.0804
Epoch [1/1], Step [100/6981], Loss: 0.7231
Epoch [1/1], Step [150/6981], Loss: 0.4373
Epoch [1/1], Step [200/6981], Loss: 0.2776
Epoch [1/1], Step [250/6981], Loss: 0.1321
Epoch [1/1], Step [300/6981], Loss: 0.1396
Epoch [1/1], Step [350/6981], Loss: 0.1790
Epoch [1/1], Step [400/6981], Loss: 0.1226
Epoch [1/1], Step [450/6981], Loss: 0.1488
Epoch [1/1], Step [500/6981], Loss: 0.1429
Epoch [1/1], Step [550/6981], Loss: 0.1590
Epoch [1/1], Step [600/6981], Loss: 0.1474
Epoch [1/1], Step [650/6981], Loss: 0.0385
Epoch [1/1], Step [700/6981], Loss: 0.1629
Epoch [1/1], Step [750/6981], Loss: 0.0536
Epoch [1/1], Step [800/6981], Loss: 0.1749
Epoch [1/1], Step [850/6981], Loss: 0.0619
Epoch [1/1], Step [900/6981], Loss: 0.1592
Epoch [1/1], Step [950/6981], Loss: 0.0727
Epoch [1/1], Step [1000/6981], Loss: 0.2276
Epoch [1/1], Step [1050/6981], Loss: 0.0467
Epoch [1/1], Step [1100/6981], Loss: 0.0715
Epoch [1/1], Step [1150/6981], Loss: 0.0255
Epoch [1

saved to colab files


In [0]:
from google.colab import files

grid_crop_size=225
patch_crop_size=64

file_name = F"relative_position_split{split}_epoch{num_epochs}_batch{batch_size}_patch_resize{resize}.pth"
print(file_name)

PATH=F"/content/gdrive/My Drive/summerthesis/saved_model/{file_name}" 
#torch.save(model.state_dict(), file_name)
#print('saved model to colab')
#files.download(file_name)
#print('saved model to local pc')

torch.save(model.state_dict(), PATH)
print('saved  model to google drive')



In [1]:
num_classe=22
file_name_p_set = F"permutation_set_{num_classe}.pt"
file_name_p_set


'permutation_set_22.pt'